<a href="https://colab.research.google.com/github/zxcasd945/Pytorchcolab/blob/main/05_PyTorch_Going_Modular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Get Data

import os
import requests
import zipfile
from pathlib import Path

data_path = Path('data/')
image_path = data_path/'pizza_steak_sushi'

if image_path.is_dir():
  print(f'{image_path} already exist.')
else:
  image_path.mkdir(parents = True, exist_ok=True)

with open(data_path/'pizza_steak_sushi.zip', 'wb') as f:
  request=requests.get('https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip')
  print('Downloading pictures now...')
  f.write(request.content)
with zipfile.ZipFile(data_path/'pizza_steak_sushi.zip','r') as zip:
  print('Unzipping data now...')
  zip.extractall(image_path)

os.remove(data_path/'pizza_steak_sushi.zip')

data/pizza_steak_sushi already exist.
Unzipping data now...


In [4]:
# Create Datasets and dataloader (data_setup.py)

%%writefile going_modular/data_setup.py

import os
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()
transform=transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.ToTensor()])

def create_dataloader(train_dir,
                      test_dir,
                      transform,
                      batch_size,
                      num_workers: int=NUM_WORKERS):

  train_data = datasets.ImageFolder(train_dir, transform = transform)
  test_data = datasets.ImageFolder(test_dir, transform = transform)

  train_dataloader = DataLoader(train_data,
                                batch_size = batch_size,
                                shuffle = True,
                                num_workers = num_workers,
                                pin_memory = True)
  test_dataloader = DataLoader(test_data,
                                batch_size = batch_size,
                                shuffle = True,
                                num_workers = num_workers,
                                pin_memory = True)
  class_names = train_data.classes

  return train_dataloader, test_dataloader, class_names






Writing going_modular/data_setup.py


In [5]:
from going_modular import data_setup
from torchvision import datasets, transforms
train_dir = 'data/pizza_steak_sushi/train'
test_dir = 'data/pizza_steak_sushi/test'
transform=transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.ToTensor()])
train_dataloader, test_dataloader,class_names = data_setup.create_dataloader(train_dir,
                                                                             test_dir,
                                                                             transform=transform,
                                                                             batch_size=25,
                                                                             num_workers=os.cpu_count())
next(iter(train_dataloader))[1]

tensor([0, 1, 1, 0, 1, 2, 2, 2, 0, 0, 2, 2, 2, 2, 1, 1, 0, 2, 1, 1, 0, 1, 2, 2,
        1])

In [6]:
%%writefile going_modular/model_builder.py
"""
Contains PyTorch model code to instantiate a TinyVGG model.
"""
import torch
from torch import nn

class TinyVGG(nn.Module):
    """Creates the TinyVGG architecture.
    Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
    See the original architecture here: https://poloclub.github.io/cnn-explainer/
    Args:
    input_shape: An integer indicating number of input channels.
    hidden_units: An integer indicating number of hidden units between layers.
    output_shape: An integer indicating number of output units.
    """
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
          nn.Conv2d(in_channels=input_shape,
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=0),
          nn.ReLU(),
          nn.Conv2d(in_channels=hidden_units,
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=0),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,
                        stride=2)
        )
        self.conv_block_2 = nn.Sequential(
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
          nn.Flatten(),
          # Where did this in_features shape come from?
          # It's because each layer of our network compresses and changes the shape of our inputs data.
          nn.Linear(in_features=hidden_units*13*13,
                    out_features=output_shape)
        )

    def forward(self, x):
        x = self.conv_block_1(x)
        x = self.conv_block_2(x)
        x = self.classifier(x)
        return x
        # return self.classifier(self.block_2(self.block_1(x))) # <- leverage the benefits of operator fusion


Writing going_modular/model_builder.py


In [7]:
from going_modular import model_builder
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = model_builder.TinyVGG(input_shape=3,
                hidden_units= 10,
                output_shape= 3).to(device)
model

TinyVGG(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1690, out_features=3, bias=True)
  )
)

In [8]:
#Create train_step, test_step, and train() in function (engine.py)
%%writefile going_modular/engine.py
import torch

def train_step(model,
               dataloader,
               loss_fn,
               optimizer,
               device):
  model.train()
  train_loss, train_acc = 0,0
  for batch, (X,y) in enumerate(dataloader):
    X,y = X.to(device), y.to(device)
    y_pred = model(X)
    loss = loss_fn(y_pred, y)
    train_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
    train_acc += (y_pred_class == y).sum().item()/len(y_pred)
  train_loss = train_loss/ len(dataloader)
  train_acc = train_acc/ len(dataloader)
  return train_loss, train_acc

def test_step(model,
               dataloader,
               loss_fn,
               device):
  model.eval()
  test_loss, test_acc = 0,0
  with torch.inference_mode():
    for batch, (X,y) in enumerate(dataloader):
      X,y = X.to(device), y.to(device)
      test_pred_logit= model(X)
      loss = loss_fn(test_pred_logit, y)
      test_loss += loss.item()
      y_pred_label = test_pred_logit.argmax(dim=1)
      test_acc += (y_pred_label == y).sum().item()/len(y_pred_label)
  test_loss = test_loss/ len(dataloader)
  test_acc = test_acc/ len(dataloader)

  return test_loss, test_acc

from tqdm.auto import tqdm

def train(model,
          train_dataloader,
          test_dataloader,
          loss_fn,
          optimizer,
          epochs,
          device):
  results = {'train_loss':[], 'train_acc':[], 'test_loss':[], 'test_acc':[]}
  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(model = model,
             dataloader = train_dataloader,
             loss_fn = loss_fn,
             optimizer = optimizer,
             device = device)
    test_loss, test_acc = test_step(model = model,
            dataloader = test_dataloader,
            loss_fn = loss_fn,
            device = device)
    print(f'Epoch:{epoch}, Train loss:{train_loss}, Train acc:{train_acc}, Test loss:{test_loss}, Test_acc:{test_acc}')
    results['train_loss'].append(train_loss)
    results['train_acc'].append(train_acc)
    results['test_loss'].append(test_loss)
    results['test_acc'].append(test_acc)

  return results

Writing going_modular/engine.py


In [9]:
from going_modular import engine, data_setup
import torch
from torch import nn
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = model.parameters(),
                             lr=0.001)
transform=transforms.Compose([transforms.Resize((64,64)),
                              transforms.ToTensor()])
EPOCHS = 5
train_dir = 'data/pizza_steak_sushi/train'
test_dir = 'data/pizza_steak_sushi/test'
train_dataloader, test_dataloader,class_names = data_setup.create_dataloader(train_dir,
                                                                             test_dir,
                                                                             transform=transform,
                                                                             batch_size=25,
                                                                             num_workers=os.cpu_count())
next(iter(train_dataloader))[1]


tensor([0, 1, 2, 1, 0, 2, 0, 1, 1, 0, 1, 2, 1, 1, 1, 2, 0, 0, 0, 0, 1, 1, 2, 1,
        0])

In [10]:
#A function to save the model(utils.py)

%%writefile going_modular/utils.py

import torch
from pathlib import Path

def save_model(model,
               target_dir,
               model_name):
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True, exist_ok = True)
  assert model_name.endwith('.pth') or model_name.endwith('.pt'),'model name should end with .pth or .pt'
  model_save_path = target_dir_path/model_name
  print(f'[INFO]: Saving model to {model_save_path}')
  torch.save(obj=model.state_dict(), f=model_save_path)

Writing going_modular/utils.py


In [11]:
%%writefile train.py

import os
import torch
from torch import nn
from torchvision import transforms
from going_modular import engine, data_setup, model_builder, utils
import argparse

parser = argparse.ArgumentParser(description='Get hyperparams')
parser.add_argument('--batch_size', default = 25, type=int, help='The number of samples per batch')
parser.add_argument('--num_epochs', default = 10, type = int, help='The number of epochs')
parser.add_argument('--hidden_units', default = 10, type = int, help = 'The number of hidden units per layer')
parser.add_argument('--lr', default= 0.001, type = float, help='Learning rate')

args = parser.parse_known_args()[0]
BATCH_SIZE = args.batch_size
NUM_EPOCHS = args.num_epochs
HIDDEN_UNITS = args.hidden_units
LEARNING_RATE = args.lr
EPOCHS = args.num_epochs

print(f'[INFO] Training model for {NUM_EPOCHS}, Batch size:{BATCH_SIZE}, Hidden units: {HIDDEN_UNITS}, Learning rate:{LEARNING_RATE}')

train_dir = 'data/pizza_steak_sushi/train'
test_dir = 'data/pizza_steak_sushi/test'

device = 'cuda' if torch.cuda.is_available() else 'cpu'

loss_fn = nn.CrossEntropyLoss()
transform=transforms.Compose([transforms.Resize((64,64)),
                              transforms.ToTensor()])
train_dataloader, test_dataloader,class_names = data_setup.create_dataloader(train_dir,
                                                                             test_dir,
                                                                             transform=transform,
                                                                             batch_size=BATCH_SIZE,
                                                                             num_workers=os.cpu_count())
model = model_builder.TinyVGG(input_shape=3,
                hidden_units= HIDDEN_UNITS,
                output_shape=len(class_names)).to(device)
optimizer = torch.optim.Adam(params = model.parameters(),
                             lr=LEARNING_RATE)






engine.train(model=model,
          train_dataloader = train_dataloader,
          test_dataloader = test_dataloader,
          loss_fn=loss_fn,
          optimizer=optimizer,
          epochs=EPOCHS,
          device=device)


Writing train.py


In [12]:
!python train.py --num_epochs 5 --batch_size 128 --hidden_units 128 --learning_rate 0.0003

[INFO] Training model for 5, Batch size:128, Hidden units: 128, Learning rate:0.001
  0% 0/5 [00:00<?, ?it/s]Epoch:0, Train loss:1.1456499099731445, Train acc:0.34604542525773196, Test loss:1.0994929075241089, Test_acc:0.25333333333333335
 20% 1/5 [00:02<00:11,  2.91s/it]Epoch:1, Train loss:1.1010570526123047, Train acc:0.41132409793814434, Test loss:1.1004853248596191, Test_acc:0.25333333333333335
 40% 2/5 [00:05<00:07,  2.63s/it]Epoch:2, Train loss:1.0996432900428772, Train acc:0.38587306701030927, Test loss:1.0850486755371094, Test_acc:0.37333333333333335
 60% 3/5 [00:06<00:04,  2.15s/it]Epoch:3, Train loss:1.0720674395561218, Train acc:0.46915270618556704, Test loss:1.0747480392456055, Test_acc:0.38666666666666666
 80% 4/5 [00:08<00:01,  1.92s/it]Epoch:4, Train loss:1.0301424264907837, Train acc:0.49585212628865977, Test loss:1.106109619140625, Test_acc:0.36
100% 5/5 [00:10<00:00,  2.01s/it]
